In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
from tqdm.auto import tqdm

class Analysis:
    def __init__(self, data: pd.DataFrame, detailed_data: pd.DataFrame):
        self.detailed_data = detailed_data
        self.data = data
        
    def plot_trajectory(self, lane_id: tuple[str, str], save_path: str = None) -> None:
        """
        Plot time-space diagram (trajectory) for vehicles on a specific lane.
        lane_id: tuple of (path_from, path_to) representing the edge/lane
        """
        df = self.detailed_data
        
        # Filter data for the specific lane
        lane_data = df[(df["path_from"] == lane_id[0]) & (df["path_to"] == lane_id[1])]
        
        if lane_data.empty:
            print(f"No data found for lane {lane_id}")
            return
        
        plt.figure(figsize=(12, 6))
        
        # Plot trajectory for each player on this lane
        for player_id in lane_data["player_id"].unique():
            player_data = lane_data[lane_data["player_id"] == player_id].sort_values("time")
            plt.plot(player_data["time"], player_data["block"], 
                    marker='o', markersize=2, alpha=0.7, linewidth=1)
        
        plt.xlabel("Time (s)")
        plt.ylabel("Block Position")
        plt.title(f"Time-Space Diagram for Lane ({lane_id[0]} → {lane_id[1]})")
        plt.gca().invert_yaxis()  # Block 0 is at intersection, so invert for intuitive view
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
    
    def plot_wait_time_distribution(self, save_path: str = None) -> None:
        """
        Plot the distribution of wait times for all completed trips.
        """
        df = self.data
        
        if df.empty:
            print("No trip data available")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Histogram of wait times
        axes[0].hist(df["wait_time"], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        axes[0].set_xlabel("Wait Time (time units)")
        axes[0].set_ylabel("Frequency")
        axes[0].set_title("Distribution of Wait Times")
        axes[0].axvline(df["wait_time"].mean(), color='red', linestyle='--', 
                       label=f'Mean: {df["wait_time"].mean():.2f}')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Box plot
        axes[1].boxplot(df["wait_time"], vert=True)
        axes[1].set_ylabel("Wait Time (time units)")
        axes[1].set_title("Wait Time Box Plot")
        axes[1].grid(True, alpha=0.3)
        
        # Add statistics text
        stats_text = f"Mean: {df['wait_time'].mean():.2f}\n"
        stats_text += f"Median: {df['wait_time'].median():.2f}\n"
        stats_text += f"Std: {df['wait_time'].std():.2f}\n"
        stats_text += f"Max: {df['wait_time'].max():.2f}"
        axes[1].text(1.15, df['wait_time'].median(), stats_text, fontsize=10,
                    verticalalignment='center')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
        
        # Also show travel time distribution
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        plt.figure(figsize=(10, 5))
        plt.hist(df["travel_time"], bins=30, edgecolor='black', alpha=0.7, color='green')
        plt.xlabel("Travel Time (time units)")
        plt.ylabel("Frequency")
        plt.title("Distribution of Total Travel Times")
        plt.axvline(df["travel_time"].mean(), color='red', linestyle='--', 
                   label=f'Mean: {df["travel_time"].mean():.2f}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        if save_path:
            travel_save_path = save_path.replace('.png', '_travel_time.png')
            plt.savefig(travel_save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
    
    def plot_queue_length_over_time(self, lane_id: tuple[str, str], save_path: str = None) -> dict:
        """
        Plot cumulative arrival-departure diagram (input-output diagram) for a specific lane.
        Returns metrics dictionary for the lane.
        """
        df = self.detailed_data.copy()
        
        # Filter data for the specific lane
        lane_data = df[(df["path_from"] == lane_id[0]) & (df["path_to"] == lane_id[1])]
        
        if lane_data.empty:
            return None
        
        # Get unique players on this lane
        players_on_lane = lane_data["player_id"].unique()
        
        arrivals = []
        departures = []
        
        for player_id in players_on_lane:
            player_data = lane_data[lane_data["player_id"] == player_id].sort_values("time")
            block_0_data = player_data[player_data["block"] == 0]
            if not block_0_data.empty:
                arrival_time = block_0_data["time"].min()
                departure_time = block_0_data["time"].max()
                arrivals.append(arrival_time)
                departures.append(departure_time)
        
        if not arrivals:
            return None
        
        arrivals = sorted(arrivals)
        departures = sorted(departures)
        
        time_min = min(min(arrivals), min(departures))
        time_max = max(max(arrivals), max(departures))
        time_range = np.arange(time_min, time_max + 1)
        
        cum_arrivals = np.array([sum(1 for a in arrivals if a <= t) for t in time_range])
        cum_departures = np.array([sum(1 for d in departures if d <= t) for t in time_range])
        
        queue_length = cum_arrivals - cum_departures
        total_delay = np.trapz(queue_length, time_range)
        avg_delay_per_vehicle = total_delay / len(arrivals) if arrivals else 0
        max_queue = queue_length.max()
        
        # Store metrics
        metrics = {
            "lane_from": lane_id[0],
            "lane_to": lane_id[1],
            "total_vehicles": len(arrivals),
            "total_delay": total_delay,
            "avg_delay_per_vehicle": avg_delay_per_vehicle,
            "max_queue_length": max_queue,
            "mean_queue_length": queue_length.mean()
        }
        
        # Plot
        fig, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        axes[0].step(time_range, cum_arrivals, where='post', label='Cumulative Arrivals (A(t))', 
                    color='blue', linewidth=2)
        axes[0].step(time_range, cum_departures, where='post', label='Cumulative Departures (D(t))', 
                    color='red', linewidth=2)
        axes[0].fill_between(time_range, cum_arrivals, cum_departures, 
                            alpha=0.3, color='orange', label='Queue (Delay Area)')
        
        axes[0].set_xlabel("Time (s)")
        axes[0].set_ylabel("Cumulative Vehicle Count")
        axes[0].set_title(f"Cumulative Arrival-Departure Diagram for Lane ({lane_id[0]} → {lane_id[1]})")
        axes[0].legend(loc='upper left')
        axes[0].grid(True, alpha=0.3)
        
        stats_text = f"Total Delay: {total_delay:.0f} veh·s\n"
        stats_text += f"Avg Delay/Vehicle: {avg_delay_per_vehicle:.1f} s\n"
        stats_text += f"Max Queue Length: {max_queue} veh\n"
        stats_text += f"Total Vehicles: {len(arrivals)}"
        axes[0].text(0.98, 0.02, stats_text, transform=axes[0].transAxes, fontsize=10,
                    verticalalignment='bottom', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        axes[1].fill_between(time_range, queue_length, alpha=0.5, color='green', label='Queue Length')
        axes[1].plot(time_range, queue_length, color='darkgreen', linewidth=1.5)
        axes[1].axhline(y=queue_length.mean(), color='red', linestyle='--', 
                       label=f'Mean Queue: {queue_length.mean():.1f}')
        
        axes[1].set_xlabel("Time (s)")
        axes[1].set_ylabel("Queue Length (vehicles)")
        axes[1].set_title(f"Queue Length Over Time for Lane ({lane_id[0]} → {lane_id[1]})")
        axes[1].legend(loc='upper right')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
        
        return metrics

    def run_full_network_analysis(self, output_dir: str = "analysis_results") -> dict:
        """
        Run comprehensive analysis on all lanes in the network.
        Generates trajectory plots, queue plots, and summary statistics for all lanes.
        Saves all plots and data to the specified output directory.
        
        Parameters:
        -----------
        output_dir : str
            Directory path where all outputs will be saved
            
        Returns:
        --------
        dict : Dictionary containing all summary statistics and dataframes
        """
        # Create output directories
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = os.path.join(output_dir, f"analysis_{timestamp}")
        trajectory_dir = os.path.join(base_dir, "trajectories")
        queue_dir = os.path.join(base_dir, "queue_analysis")
        summary_dir = os.path.join(base_dir, "summary")
        
        for dir_path in [trajectory_dir, queue_dir, summary_dir]:
            os.makedirs(dir_path, exist_ok=True)
        
        print(f"📁 Output directory: {base_dir}")
        print("=" * 60)
        
        # Get all unique lanes from detailed data
        lanes = self.detailed_data.groupby(["path_from", "path_to"]).size().reset_index(name='count')
        lanes = lanes.sort_values('count', ascending=False)
        total_lanes = len(lanes)
        
        print(f"🔍 Found {total_lanes} lanes with vehicle data")
        print("=" * 60)
        
        # Store all lane metrics
        lane_metrics_list = []
        
        # Process each lane
        for idx, row in tqdm(lanes.iterrows(), total=total_lanes, desc="Processing lanes"):
            lane_id = (row["path_from"], row["path_to"])
            lane_str = f"{lane_id[0]}_to_{lane_id[1]}"
            
            # Generate trajectory plot
            traj_path = os.path.join(trajectory_dir, f"trajectory_{lane_str}.png")
            self.plot_trajectory(lane_id=lane_id, save_path=traj_path)
            
            # Generate queue analysis plot and get metrics
            queue_path = os.path.join(queue_dir, f"queue_{lane_str}.png")
            metrics = self.plot_queue_length_over_time(lane_id=lane_id, save_path=queue_path)
            
            if metrics:
                lane_metrics_list.append(metrics)
        
        # Create lane metrics dataframe
        lane_metrics_df = pd.DataFrame(lane_metrics_list)
        
        # Calculate network-wide statistics
        network_stats = self._calculate_network_statistics()
        
        # Generate summary plots
        self._generate_summary_plots(lane_metrics_df, network_stats, summary_dir)
        
        # Save all dataframes
        self._save_dataframes(lane_metrics_df, network_stats, summary_dir)
        
        # Generate final report
        self._generate_report(lane_metrics_df, network_stats, base_dir)
        
        print("\n" + "=" * 60)
        print("✅ Analysis Complete!")
        print(f"📊 All results saved to: {base_dir}")
        print("=" * 60)
        
        return {
            "lane_metrics": lane_metrics_df,
            "network_stats": network_stats,
            "output_dir": base_dir
        }
    
    def _calculate_network_statistics(self) -> dict:
        """Calculate network-wide statistics from trip data."""
        df = self.data.copy()
        
        if df.empty:
            return {}
        
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        stats = {
            # Trip statistics
            "total_trips_completed": len(df),
            "avg_wait_time": df["wait_time"].mean(),
            "median_wait_time": df["wait_time"].median(),
            "std_wait_time": df["wait_time"].std(),
            "min_wait_time": df["wait_time"].min(),
            "max_wait_time": df["wait_time"].max(),
            "percentile_25_wait": df["wait_time"].quantile(0.25),
            "percentile_75_wait": df["wait_time"].quantile(0.75),
            "percentile_95_wait": df["wait_time"].quantile(0.95),
            
            # Travel time statistics
            "avg_travel_time": df["travel_time"].mean(),
            "median_travel_time": df["travel_time"].median(),
            "std_travel_time": df["travel_time"].std(),
            "min_travel_time": df["travel_time"].min(),
            "max_travel_time": df["travel_time"].max(),
            "percentile_25_travel": df["travel_time"].quantile(0.25),
            "percentile_75_travel": df["travel_time"].quantile(0.75),
            "percentile_95_travel": df["travel_time"].quantile(0.95),
            
            # Throughput
            "simulation_duration": df["departed_time"].max() - df["arrived_time"].min(),
            "throughput_per_hour": len(df) / ((df["departed_time"].max() - df["arrived_time"].min()) / 3600) if df["departed_time"].max() > df["arrived_time"].min() else 0,
        }
        
        return stats
    
    def _generate_summary_plots(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Generate summary plots for the entire network."""
        
        # 1. Wait time and Travel time distribution (combined)
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        df = self.data.copy()
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        # Wait time histogram
        axes[0, 0].hist(df["wait_time"], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
        axes[0, 0].axvline(df["wait_time"].mean(), color='red', linestyle='--', linewidth=2,
                          label=f'Mean: {df["wait_time"].mean():.1f}')
        axes[0, 0].axvline(df["wait_time"].median(), color='orange', linestyle='-.', linewidth=2,
                          label=f'Median: {df["wait_time"].median():.1f}')
        axes[0, 0].set_xlabel("Wait Time (time units)", fontsize=12)
        axes[0, 0].set_ylabel("Frequency", fontsize=12)
        axes[0, 0].set_title("Network-Wide Wait Time Distribution", fontsize=14, fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Travel time histogram
        axes[0, 1].hist(df["travel_time"], bins=50, edgecolor='black', alpha=0.7, color='forestgreen')
        axes[0, 1].axvline(df["travel_time"].mean(), color='red', linestyle='--', linewidth=2,
                          label=f'Mean: {df["travel_time"].mean():.1f}')
        axes[0, 1].axvline(df["travel_time"].median(), color='orange', linestyle='-.', linewidth=2,
                          label=f'Median: {df["travel_time"].median():.1f}')
        axes[0, 1].set_xlabel("Travel Time (time units)", fontsize=12)
        axes[0, 1].set_ylabel("Frequency", fontsize=12)
        axes[0, 1].set_title("Network-Wide Travel Time Distribution", fontsize=14, fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Combined box plots
        box_data = [df["wait_time"].dropna(), df["travel_time"].dropna()]
        bp = axes[1, 0].boxplot(box_data, labels=['Wait Time', 'Travel Time'], patch_artist=True)
        colors = ['steelblue', 'forestgreen']
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        axes[1, 0].set_ylabel("Time (time units)", fontsize=12)
        axes[1, 0].set_title("Wait Time vs Travel Time Comparison", fontsize=14, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
        
        # Arrival time vs travel time scatter
        axes[1, 1].scatter(df["arrived_time"], df["travel_time"], alpha=0.3, s=10, c='purple')
        axes[1, 1].set_xlabel("Arrival Time (time units)", fontsize=12)
        axes[1, 1].set_ylabel("Travel Time (time units)", fontsize=12)
        axes[1, 1].set_title("Travel Time vs Arrival Time", fontsize=14, fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "network_time_distributions.png"), dpi=200, bbox_inches='tight')
        plt.close()
        
        # 2. Lane-level analysis plots
        if not lane_metrics_df.empty:
            fig, axes = plt.subplots(2, 2, figsize=(16, 12))
            
            # Top 20 lanes by total delay
            top_delay = lane_metrics_df.nlargest(20, 'total_delay')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_delay.iterrows()]
            axes[0, 0].barh(range(len(top_delay)), top_delay['total_delay'], color='coral', edgecolor='black')
            axes[0, 0].set_yticks(range(len(top_delay)))
            axes[0, 0].set_yticklabels(lane_labels, fontsize=8)
            axes[0, 0].set_xlabel("Total Delay (veh·s)", fontsize=12)
            axes[0, 0].set_title("Top 20 Lanes by Total Delay", fontsize=14, fontweight='bold')
            axes[0, 0].grid(True, alpha=0.3, axis='x')
            axes[0, 0].invert_yaxis()
            
            # Top 20 lanes by max queue
            top_queue = lane_metrics_df.nlargest(20, 'max_queue_length')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_queue.iterrows()]
            axes[0, 1].barh(range(len(top_queue)), top_queue['max_queue_length'], color='teal', edgecolor='black')
            axes[0, 1].set_yticks(range(len(top_queue)))
            axes[0, 1].set_yticklabels(lane_labels, fontsize=8)
            axes[0, 1].set_xlabel("Max Queue Length (vehicles)", fontsize=12)
            axes[0, 1].set_title("Top 20 Lanes by Max Queue Length", fontsize=14, fontweight='bold')
            axes[0, 1].grid(True, alpha=0.3, axis='x')
            axes[0, 1].invert_yaxis()
            
            # Vehicle count distribution across lanes
            axes[1, 0].hist(lane_metrics_df['total_vehicles'], bins=30, edgecolor='black', alpha=0.7, color='goldenrod')
            axes[1, 0].axvline(lane_metrics_df['total_vehicles'].mean(), color='red', linestyle='--', linewidth=2,
                              label=f'Mean: {lane_metrics_df["total_vehicles"].mean():.1f}')
            axes[1, 0].set_xlabel("Number of Vehicles", fontsize=12)
            axes[1, 0].set_ylabel("Number of Lanes", fontsize=12)
            axes[1, 0].set_title("Distribution of Vehicle Counts per Lane", fontsize=14, fontweight='bold')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
            
            # Scatter: vehicles vs avg delay
            axes[1, 1].scatter(lane_metrics_df['total_vehicles'], lane_metrics_df['avg_delay_per_vehicle'], 
                              alpha=0.6, s=50, c='darkviolet', edgecolors='black', linewidth=0.5)
            axes[1, 1].set_xlabel("Total Vehicles", fontsize=12)
            axes[1, 1].set_ylabel("Average Delay per Vehicle (s)", fontsize=12)
            axes[1, 1].set_title("Vehicle Volume vs Average Delay per Vehicle", fontsize=14, fontweight='bold')
            axes[1, 1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, "lane_level_analysis.png"), dpi=200, bbox_inches='tight')
            plt.close()
        
        # 3. Time series analysis
        fig, axes = plt.subplots(2, 1, figsize=(16, 10))
        
        # Arrivals over time
        arrival_counts = df.groupby('arrived_time').size()
        window = 60  # 1 minute rolling window
        arrival_rate = arrival_counts.rolling(window=window, min_periods=1).sum()
        
        axes[0].plot(arrival_rate.index, arrival_rate.values, color='blue', linewidth=1.5, alpha=0.8)
        axes[0].fill_between(arrival_rate.index, arrival_rate.values, alpha=0.3, color='blue')
        axes[0].set_xlabel("Time (s)", fontsize=12)
        axes[0].set_ylabel(f"Arrivals (per {window}s window)", fontsize=12)
        axes[0].set_title("Vehicle Arrival Rate Over Time", fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        
        # Departures over time
        departure_counts = df.groupby('departed_time').size()
        departure_rate = departure_counts.rolling(window=window, min_periods=1).sum()
        
        axes[1].plot(departure_rate.index, departure_rate.values, color='green', linewidth=1.5, alpha=0.8)
        axes[1].fill_between(departure_rate.index, departure_rate.values, alpha=0.3, color='green')
        axes[1].set_xlabel("Time (s)", fontsize=12)
        axes[1].set_ylabel(f"Departures (per {window}s window)", fontsize=12)
        axes[1].set_title("Vehicle Departure Rate Over Time", fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "temporal_analysis.png"), dpi=200, bbox_inches='tight')
        plt.close()
        
        # 4. Comprehensive dashboard
        self._generate_dashboard(lane_metrics_df, network_stats, output_dir)
    
    def _generate_dashboard(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Generate a comprehensive dashboard summary."""
        fig = plt.figure(figsize=(20, 16))
        
        # Create grid for dashboard
        gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
        
        df = self.data.copy()
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        # Title
        fig.suptitle("🚗 Traffic Simulation Analysis Dashboard", fontsize=20, fontweight='bold', y=0.98)
        
        # 1. Key metrics boxes (top row)
        ax_metrics = fig.add_subplot(gs[0, :])
        ax_metrics.axis('off')
        
        metrics_text = (
            f"📊 NETWORK SUMMARY STATISTICS\n\n"
            f"Total Trips Completed: {network_stats.get('total_trips_completed', 0):,}\n"
            f"Average Wait Time: {network_stats.get('avg_wait_time', 0):.2f} s\n"
            f"Average Travel Time: {network_stats.get('avg_travel_time', 0):.2f} s\n"
            f"95th Percentile Wait Time: {network_stats.get('percentile_95_wait', 0):.2f} s\n"
            f"Throughput: {network_stats.get('throughput_per_hour', 0):.0f} veh/hour\n"
            f"Total Lanes Analyzed: {len(lane_metrics_df)}"
        )
        
        ax_metrics.text(0.5, 0.5, metrics_text, transform=ax_metrics.transAxes, fontsize=14,
                       verticalalignment='center', horizontalalignment='center',
                       bbox=dict(boxstyle='round,pad=1', facecolor='lightblue', alpha=0.8),
                       family='monospace')
        
        # 2. Wait time distribution
        ax1 = fig.add_subplot(gs[1, 0:2])
        ax1.hist(df["wait_time"], bins=40, edgecolor='black', alpha=0.7, color='steelblue')
        ax1.axvline(df["wait_time"].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        ax1.set_xlabel("Wait Time (s)")
        ax1.set_ylabel("Frequency")
        ax1.set_title("Wait Time Distribution", fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 3. Travel time distribution
        ax2 = fig.add_subplot(gs[1, 2:4])
        ax2.hist(df["travel_time"], bins=40, edgecolor='black', alpha=0.7, color='forestgreen')
        ax2.axvline(df["travel_time"].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        ax2.set_xlabel("Travel Time (s)")
        ax2.set_ylabel("Frequency")
        ax2.set_title("Travel Time Distribution", fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # 4. Top 10 congested lanes
        ax3 = fig.add_subplot(gs[2, 0:2])
        if not lane_metrics_df.empty:
            top_delay = lane_metrics_df.nlargest(10, 'total_delay')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_delay.iterrows()]
            bars = ax3.barh(range(len(top_delay)), top_delay['total_delay'], color='coral', edgecolor='black')
            ax3.set_yticks(range(len(top_delay)))
            ax3.set_yticklabels(lane_labels, fontsize=9)
            ax3.set_xlabel("Total Delay (veh·s)")
            ax3.set_title("Top 10 Most Congested Lanes", fontweight='bold')
            ax3.invert_yaxis()
            ax3.grid(True, alpha=0.3, axis='x')
        
        # 5. Top 10 lanes by queue
        ax4 = fig.add_subplot(gs[2, 2:4])
        if not lane_metrics_df.empty:
            top_queue = lane_metrics_df.nlargest(10, 'max_queue_length')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_queue.iterrows()]
            ax4.barh(range(len(top_queue)), top_queue['max_queue_length'], color='teal', edgecolor='black')
            ax4.set_yticks(range(len(top_queue)))
            ax4.set_yticklabels(lane_labels, fontsize=9)
            ax4.set_xlabel("Max Queue Length (vehicles)")
            ax4.set_title("Top 10 Lanes by Max Queue", fontweight='bold')
            ax4.invert_yaxis()
            ax4.grid(True, alpha=0.3, axis='x')
        
        # 6. Arrival/Departure rates over time
        ax5 = fig.add_subplot(gs[3, 0:2])
        window = 120
        arrival_counts = df.groupby('arrived_time').size()
        departure_counts = df.groupby('departed_time').size()
        
        all_times = pd.Index(range(int(df['arrived_time'].min()), int(df['departed_time'].max()) + 1))
        arrival_rate = arrival_counts.reindex(all_times, fill_value=0).rolling(window=window, min_periods=1).sum()
        departure_rate = departure_counts.reindex(all_times, fill_value=0).rolling(window=window, min_periods=1).sum()
        
        ax5.plot(arrival_rate.index, arrival_rate.values, color='blue', linewidth=1.5, label='Arrivals', alpha=0.8)
        ax5.plot(departure_rate.index, departure_rate.values, color='green', linewidth=1.5, label='Departures', alpha=0.8)
        ax5.fill_between(arrival_rate.index, arrival_rate.values, departure_rate.values, 
                        where=arrival_rate.values > departure_rate.values, alpha=0.3, color='red', label='Queue Building')
        ax5.set_xlabel("Time (s)")
        ax5.set_ylabel(f"Vehicles (per {window}s)")
        ax5.set_title("Arrival vs Departure Rate", fontweight='bold')
        ax5.legend(loc='upper right', fontsize=8)
        ax5.grid(True, alpha=0.3)
        
        # 7. Cumulative trips
        ax6 = fig.add_subplot(gs[3, 2:4])
        cum_arrivals = arrival_counts.reindex(all_times, fill_value=0).cumsum()
        cum_departures = departure_counts.reindex(all_times, fill_value=0).cumsum()
        
        ax6.plot(cum_arrivals.index, cum_arrivals.values, color='blue', linewidth=2, label='Cumulative Arrivals')
        ax6.plot(cum_departures.index, cum_departures.values, color='green', linewidth=2, label='Cumulative Departures')
        ax6.fill_between(cum_arrivals.index, cum_arrivals.values, cum_departures.values, alpha=0.3, color='orange')
        ax6.set_xlabel("Time (s)")
        ax6.set_ylabel("Cumulative Vehicles")
        ax6.set_title("Cumulative Arrivals vs Departures", fontweight='bold')
        ax6.legend(loc='upper left')
        ax6.grid(True, alpha=0.3)
        
        plt.savefig(os.path.join(output_dir, "comprehensive_dashboard.png"), dpi=200, bbox_inches='tight')
        plt.close()
    
    def _save_dataframes(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Save all analysis dataframes to CSV."""
        
        # 1. Lane metrics
        lane_metrics_df.to_csv(os.path.join(output_dir, "lane_metrics.csv"), index=False)
        
        # 2. Network statistics
        stats_df = pd.DataFrame([network_stats])
        stats_df.to_csv(os.path.join(output_dir, "network_statistics.csv"), index=False)
        
        # 3. Trip-level data with travel time
        df = self.data.copy()
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        df.to_csv(os.path.join(output_dir, "trip_data_with_travel_time.csv"), index=False)
        
        # 4. Detailed movement data
        self.detailed_data.to_csv(os.path.join(output_dir, "detailed_movement_data.csv"), index=False)
        
        # 5. Summary by lane
        lane_summary = self.detailed_data.groupby(['path_from', 'path_to']).agg({
            'player_id': 'nunique',
            'time': ['min', 'max'],
            'block': ['min', 'max', 'mean']
        }).reset_index()
        lane_summary.columns = ['lane_from', 'lane_to', 'unique_vehicles', 
                               'first_observation', 'last_observation',
                               'min_block', 'max_block', 'avg_block']
        lane_summary.to_csv(os.path.join(output_dir, "lane_summary.csv"), index=False)
        
        print(f"📄 Saved CSV files to {output_dir}")
    
    def _generate_report(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Generate a text report summarizing the analysis."""
        
        report_lines = [
            "=" * 80,
            "TRAFFIC SIMULATION ANALYSIS REPORT",
            f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            "=" * 80,
            "",
            "NETWORK-WIDE STATISTICS",
            "-" * 40,
            f"Total Trips Completed: {network_stats.get('total_trips_completed', 0):,}",
            f"Simulation Duration: {network_stats.get('simulation_duration', 0):.0f} seconds",
            f"Throughput: {network_stats.get('throughput_per_hour', 0):.0f} vehicles/hour",
            "",
            "WAIT TIME STATISTICS",
            "-" * 40,
            f"Average Wait Time: {network_stats.get('avg_wait_time', 0):.2f} seconds",
            f"Median Wait Time: {network_stats.get('median_wait_time', 0):.2f} seconds",
            f"Std Dev Wait Time: {network_stats.get('std_wait_time', 0):.2f} seconds",
            f"Min Wait Time: {network_stats.get('min_wait_time', 0):.2f} seconds",
            f"Max Wait Time: {network_stats.get('max_wait_time', 0):.2f} seconds",
            f"25th Percentile: {network_stats.get('percentile_25_wait', 0):.2f} seconds",
            f"75th Percentile: {network_stats.get('percentile_75_wait', 0):.2f} seconds",
            f"95th Percentile: {network_stats.get('percentile_95_wait', 0):.2f} seconds",
            "",
            "TRAVEL TIME STATISTICS",
            "-" * 40,
            f"Average Travel Time: {network_stats.get('avg_travel_time', 0):.2f} seconds",
            f"Median Travel Time: {network_stats.get('median_travel_time', 0):.2f} seconds",
            f"Std Dev Travel Time: {network_stats.get('std_travel_time', 0):.2f} seconds",
            f"Min Travel Time: {network_stats.get('min_travel_time', 0):.2f} seconds",
            f"Max Travel Time: {network_stats.get('max_travel_time', 0):.2f} seconds",
            f"25th Percentile: {network_stats.get('percentile_25_travel', 0):.2f} seconds",
            f"75th Percentile: {network_stats.get('percentile_75_travel', 0):.2f} seconds",
            f"95th Percentile: {network_stats.get('percentile_95_travel', 0):.2f} seconds",
            "",
            "LANE-LEVEL STATISTICS",
            "-" * 40,
            f"Total Lanes Analyzed: {len(lane_metrics_df)}",
        ]
        
        if not lane_metrics_df.empty:
            report_lines.extend([
                f"Total Network Delay: {lane_metrics_df['total_delay'].sum():.0f} vehicle-seconds",
                f"Average Delay per Lane: {lane_metrics_df['total_delay'].mean():.2f} vehicle-seconds",
                f"Max Single Lane Delay: {lane_metrics_df['total_delay'].max():.0f} vehicle-seconds",
                f"Average Max Queue Length: {lane_metrics_df['max_queue_length'].mean():.2f} vehicles",
                f"Highest Max Queue: {lane_metrics_df['max_queue_length'].max():.0f} vehicles",
                "",
                "TOP 10 MOST CONGESTED LANES (by Total Delay)",
                "-" * 40,
            ])
            
            top_10 = lane_metrics_df.nlargest(10, 'total_delay')
            for i, (_, row) in enumerate(top_10.iterrows(), 1):
                report_lines.append(
                    f"{i:2}. Lane {row['lane_from']} → {row['lane_to']}: "
                    f"Delay={row['total_delay']:.0f}s, MaxQueue={row['max_queue_length']:.0f}veh, "
                    f"Vehicles={row['total_vehicles']:.0f}"
                )
        
        report_lines.extend([
            "",
            "=" * 80,
            "OUTPUT FILES",
            "-" * 40,
            "- lane_metrics.csv: Metrics for each lane",
            "- network_statistics.csv: Network-wide statistics",
            "- trip_data_with_travel_time.csv: All trip records",
            "- detailed_movement_data.csv: Vehicle movement logs",
            "- lane_summary.csv: Summary statistics per lane",
            "- comprehensive_dashboard.png: Visual dashboard",
            "- network_time_distributions.png: Time distribution plots",
            "- lane_level_analysis.png: Lane-level analysis plots",
            "- temporal_analysis.png: Time series analysis",
            "- trajectories/: Individual lane trajectory plots",
            "- queue_analysis/: Individual lane queue plots",
            "=" * 80,
        ])
        
        report_text = "\n".join(report_lines)
        
        with open(os.path.join(output_dir, "analysis_report.txt"), 'w') as f:
            f.write(report_text)
        
        print("\n" + report_text)

# Session 7: Complete Guide to SUMO and sumolib

## Table of Contents
1. [Introduction to SUMO and sumolib](#introduction)
2. [Setup and Installation](#setup)
3. [Importing sumolib](#importing)
4. [Creating and Loading Networks](#networks)
5. [Network Properties and Methods](#network-properties)
6. [Network Elements (Nodes, Edges, Lanes)](#elements)
7. [Practical Network Analysis Examples](#practical-examples)
8. [Route and Vehicle Generation](#routes)
9. [Parsing Route Files](#parsing-routes)
10. [Network Calculations](#calculations)
11. [Network Selection](#selection)
12. [Simulation Configuration](#config)
13. [TraCI Integration - Basics](#traci-basics)
14. [TraCI - Basic Workflow](#traci-workflow)
15. [TraCI - Simulation Control](#traci-control)
16. [TraCI - Adding Vehicles Dynamically](#traci-add-vehicles)
17. [TraCI - Subscriptions](#traci-subscriptions)
18. [TraCI - Context Subscriptions](#traci-context)
19. [TraCI - Subscription Filters](#traci-filters)
20. [TraCI - Vehicle Information](#traci-vehicle-info)
21. [TraCI - Vehicle Control](#traci-vehicle-control)
22. [Advanced Topics - Parallel Simulations](#parallel)
23. [Advanced Topics - Error Handling](#error-handling)
24. [Advanced Topics - Best Practices](#best-practices)
25. [Complete Example - Traffic Light Control](#complete-example)

---

## 1. Introduction to SUMO and sumolib <a name="introduction"></a>

**SUMO** (Simulation of Urban MObility) is an open source, highly portable, microscopic and continuous traffic simulation package designed to handle large networks.

**sumolib** is a set of Python modules for working with SUMO networks, simulation output, and other simulation artifacts. It provides:
- Network analysis and manipulation
- XML parsing utilities
- Coordinate transformations
- Route analysis
- Output processing
- Geometric calculations

**Key Features:**
- Load and analyze SUMO network files (`.net.xml`)
- Parse various SUMO XML files (routes, outputs, configurations)
- Perform coordinate transformations (geo, network, lane coordinates)
- Analyze traffic data and statistics
- Integrate with TraCI for runtime simulation control

**Documentation:**
- Official sumolib docs: https://sumo.dlr.de/docs/Tools/Sumolib.html
- PyDoc API reference: https://sumo.dlr.de/pydoc/sumolib.html
- TraCI interface: https://sumo.dlr.de/docs/TraCI/Interfacing_TraCI_from_Python.html

---

## 2. Setup and Installation <a name="setup"></a>

### Environment Setup

Before using sumolib, you need to ensure SUMO is installed and the `SUMO_HOME` environment variable is set correctly.

In [2]:
import os
import sys

from sumolib import checkBinary
print(checkBinary("sumo"))
print(checkBinary("netconvert"))


/usr/share/sumo/bin/sumo
/usr/share/sumo/bin/netconvert


In [3]:
import os
import sys

# Check if SUMO_HOME is set
if 'SUMO_HOME' in os.environ:
    sumo_home = os.environ['SUMO_HOME']
    print(f"✓ SUMO_HOME is set to: {sumo_home}")
    
    # Add tools to Python path
    tools_path = os.path.join(sumo_home, 'tools')
    if tools_path not in sys.path:
        sys.path.append(tools_path)
    print(f"✓ Added tools directory to path: {tools_path}")
else:
    print("✗ SUMO_HOME environment variable is not set!")
    print("  Please set it to your SUMO installation directory")
    print("  Example (Linux/Mac): export SUMO_HOME=/usr/share/sumo")
    print("  Example (Windows): set SUMO_HOME=C:\\Program Files\\SUMO")

✓ SUMO_HOME is set to: /usr/share/sumo
✓ Added tools directory to path: /usr/share/sumo/tools


---

## 3. Importing sumolib <a name="importing"></a>

The standard way to import sumolib in your scripts:

In [4]:
import sumolib
import sumolib.net  # For network operations
import sumolib.xml  # For XML parsing
import sumolib.geomhelper  # For geometric calculations
import sumolib.miscutils  # For utility functions
print("✓ Successfully imported sumolib submodules")

✓ Successfully imported sumolib submodules


---

## 4. Creating and Loading Networks <a name="networks"></a>

SUMO networks are stored in `.net.xml` files. These files contain the complete road network structure including nodes, edges, lanes, connections, and traffic lights.

### 4.1 Creating a Network File

You can create network files programmatically using custom generators:

In [5]:
from xml_generators import  SUMONetworkGenerator
import pandas as pd

network_nodes = pd.read_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/SiouxFallsXY.csv",
                            index_col=1)
# network_nodes
trafic_nodes = [str(i) for i in network_nodes.copy().index.tolist()]
# print(trafic_nodes)

network_nodes = network_nodes.apply(lambda x: {"x": float(x["x"]), "y": float(x["y"])}, axis=1).to_dict()
print("✓ Successfully loaded network nodes")
# network_nodes

network_edges = pd.read_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/SiouxFallsNetwork.csv")
# network_edges
network_edges = list(network_edges.apply(lambda x: {"from": int(x["from"]), "to": int(x["to"]),
                                               "length": float(x["length"])}, axis=1).values)
print("✓ Successfully loaded network edges")
# network_edges


xml_parsing = SUMONetworkGenerator(nodes=network_nodes, 
                                   edges=network_edges, 
                                   default_speed=80/3.6)
xml_parsing.add_traffic_lights_auto(green_time=30,
                                    yellow_time=5)
xml_parsing.save_with_netconvert(r"result.net.xml")

✓ Successfully loaded network nodes
✓ Successfully loaded network edges
Validated 24 nodes and 76 edges
Added traffic light to junction J2 with 4 phases
Added traffic light to junction J3 with 6 phases
Added traffic light to junction J1 with 4 phases
Added traffic light to junction J6 with 6 phases
Added traffic light to junction J4 with 6 phases
Added traffic light to junction J12 with 6 phases
Added traffic light to junction J5 with 6 phases
Added traffic light to junction J11 with 8 phases
Added traffic light to junction J9 with 6 phases
Added traffic light to junction J8 with 8 phases
Added traffic light to junction J18 with 6 phases
Added traffic light to junction J7 with 4 phases
Added traffic light to junction J16 with 8 phases
Added traffic light to junction J10 with 10 phases
Added traffic light to junction J15 with 8 phases
Added traffic light to junction J17 with 6 phases
Added traffic light to junction J14 with 6 phases
Added traffic light to junction J13 with 4 phases
Adde

---

## 8. Route and Vehicle Generation <a name="routes"></a>

Create routes and vehicle definitions for your simulation using route generators.

In [13]:
demand_path = r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/demand1.csv"
demand_path = r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/demand2.csv"

In [14]:
from xml_generators import SUMORouteGenerator
import pandas as pd

# Create generator
route_gen = SUMORouteGenerator()

# Add vehicle type
route_gen.add_vehicle_type(type_id='car', 
                           accel=2.6, 
                           decel=4.5, 
                           length=5.0, 
                           max_speed=33.33, 
                           color="red")

# Read demand data from CSV
demand_df = pd.read_csv(demand_path)
print(f"✓ Loaded {len(demand_df)} trips from demand.csv")
print(f"Demand data preview:\n{demand_df.head()}")

# Create trips from demand data
for idx, row in demand_df.iterrows():
    trip_id = f"trip_{int(row['player_id'])}"
    from_edge = f"E{int(row['source'])}_{int(row['source']) + 1}"  # Adjust edge naming as needed
    to_edge = f"E{int(row['destination'])}_{int(row['destination']) + 1}"  # Adjust edge naming as needed
    depart_time = int(row['arrival_time'])
    
    route_gen.add_trip(trip_id=trip_id, 
                      from_edge=from_edge, 
                      to_edge=to_edge, 
                      depart=depart_time, 
                      vtype='car', 
                      color="1,0,0")

print(f"✓ Created {len(demand_df)} trips")

# Save the route file
route_gen.save_xml('routes.rou.xml', begin=0, end=3600)
print("✓ Saved routes to routes.rou.xml")

Added vehicle type 'default'
Added vehicle type 'car'
✓ Loaded 2000 trips from demand.csv
Demand data preview:
   arrival_time  source  destination  player_id
0             1       2           21          1
1             1      20            2          2
2             2      16           14          3
3             5      18            6          4
4             7       8           13          5
Added trip 'trip_1' from E2_3 to E21_22 (dynamic routing)
Added trip 'trip_2' from E20_21 to E2_3 (dynamic routing)
Added trip 'trip_3' from E16_17 to E14_15 (dynamic routing)
Added trip 'trip_4' from E18_19 to E6_7 (dynamic routing)
Added trip 'trip_5' from E8_9 to E13_14 (dynamic routing)
Added trip 'trip_6' from E15_16 to E16_17 (dynamic routing)
Added trip 'trip_7' from E15_16 to E1_2 (dynamic routing)
Added trip 'trip_8' from E4_5 to E22_23 (dynamic routing)
Added trip 'trip_9' from E16_17 to E15_16 (dynamic routing)
Added trip 'trip_10' from E8_9 to E23_24 (dynamic routing)
Added trip 'tr

---

## 11. Network Selection <a name="selection"></a>

SUMO allows you to work with subsets of the network using selection files.

In [3]:
import sumolib

# Load network
net = sumolib.net.readNet('result.net.xml')

# Create a selection programmatically FIRST
# Example: Select all edges with speed > 2.0 m/s
high_speed_edges = []
for edge in net.getEdges():
    if edge.getSpeed() > 40.0:
        high_speed_edges.append(edge.getID())

# Write selection to file
with open('high_speed_selection.txt', 'w') as f:
    for edge_id in high_speed_edges:
        f.write(f"edge:{edge_id}\n")

print(f"Created selection with {len(high_speed_edges)} high-speed edges")

# NOW load the selection file
net.loadSelection('high_speed_selection.txt')

# Calculate cumulative length of selected edges
cumul_length = 0.0
selected_count = 0

for edge in net.getEdges():
    if edge.isSelected():
        cumul_length += edge.getLength()
        selected_count += 1

print(f"\nSelected {selected_count} edges")
print(f"Total length of selected edges: {cumul_length:.2f} meters")
if selected_count > 0:
    print(f"Average edge length: {cumul_length/selected_count:.2f} meters")

Created selection with 0 high-speed edges

Selected 0 edges
Total length of selected edges: 0.00 meters


---

## 12. Simulation Configuration <a name="config"></a>

Create SUMO configuration files programmatically:

In [4]:
from xml_generators import SUMORouteGenerator
import pandas as pd

# Create generator
route_gen = SUMORouteGenerator()

# Add vehicle type
route_gen.add_vehicle_type(type_id='car', 
                           accel=2.6, 
                           decel=4.5, 
                           length=5.0, 
                           max_speed=33.33, 
                           color="red")

# Read demand data from CSV
demand_df = pd.read_csv(demand_path)
print(f"✓ Loaded {len(demand_df)} trips from demand.csv")
print(f"Demand data preview:\n{demand_df.head()}")

# Create trips from demand data using junctions (not edges)
# This allows SUMO to automatically compute the shortest path
for idx, row in demand_df.iterrows():
    trip_id = f"trip_{int(row['player_id'])}"
    from_junction = str(int(row['source']))  # Will be converted to 'J1', 'J2', etc.
    to_junction = str(int(row['destination']))
    depart_time = int(row['arrival_time'])
    
    route_gen.add_trip_from_junctions(
        trip_id=trip_id, 
        from_junction=from_junction, 
        to_junction=to_junction, 
        depart=depart_time, 
        vtype='car', 
        color="1,0,0"
    )

print(f"✓ Created {len(demand_df)} trips")

# Save the route file
route_gen.save_xml('routes.rou.xml', begin=0, end=3600)
print("✓ Saved routes to routes.rou.xml")

Added vehicle type 'default'
Added vehicle type 'car'
✓ Loaded 2000 trips from demand.csv
Demand data preview:
   arrival_time  source  destination  player_id
0             1       2           21          1
1             1      20            2          2
2             2      16           14          3
3             5      18            6          4
4             7       8           13          5
Added trip 'trip_1' from junction J2 to J21 (dynamic routing)
Added trip 'trip_2' from junction J20 to J2 (dynamic routing)
Added trip 'trip_3' from junction J16 to J14 (dynamic routing)
Added trip 'trip_4' from junction J18 to J6 (dynamic routing)
Added trip 'trip_5' from junction J8 to J13 (dynamic routing)
Added trip 'trip_6' from junction J15 to J16 (dynamic routing)
Added trip 'trip_7' from junction J15 to J1 (dynamic routing)
Added trip 'trip_8' from junction J4 to J22 (dynamic routing)
Added trip 'trip_9' from junction J16 to J15 (dynamic routing)
Added trip 'trip_10' from junction J8 to

In [5]:
import os
import sys
import pandas as pd
import csv
from collections import defaultdict

# Ensure SUMO_HOME is set
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci

# Read demand data to get trip information
demand_df = pd.read_csv(demand_path)
print(f"Loaded {len(demand_df)} trips from demand")

# Create dictionaries to store vehicle data
vehicle_data = {}  # Store data for each vehicle
detailed_records = []  # Store detailed timestep records

# Edge mapping: convert node numbers to edge IDs (adjust based on your network)
# This assumes edges are named as E{from}_{to}
def get_edge_from_node(node):
    """Convert node number to an edge ID starting from that node"""
    return f"E{node}_"  # We'll need to know the actual edges in the network

# Prepare SUMO command
sumoBinary = "sumo-gui"  # Use "sumo-gui" for visualization
sumoConfig = "simulation.sumocfg"
sumoCmd = [sumoBinary, "-c", sumoConfig, "--no-warnings", "true"]

print("Starting SUMO simulation...")

try:
    # Start simulation
    traci.start(sumoCmd)
    
    # Initialize tracking for each vehicle from demand
    for _, row in demand_df.iterrows():
        veh_id = f"trip_{int(row['player_id'])}"
        vehicle_data[veh_id] = {
            'player_id': int(row['player_id']),
            'source': int(row['source']),
            'destination': int(row['destination']),
            'departure_time': int(row['arrival_time']),
            'actual_depart': None,
            'actual_arrival': None,
            'wait_time': 0,
            'current_edge': None,
            'previous_edge': None,
        }
    
    step = 0
    departed_vehicles = set()
    arrived_vehicles = set()
    
    print(f"Simulation running...", end='')
    
    # Run simulation
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        current_time = int(traci.simulation.getTime())
        
        # Get list of currently running vehicles
        vehicle_ids = traci.vehicle.getIDList()
        
        # Track departed vehicles
        newly_departed = traci.simulation.getDepartedIDList()
        for veh_id in newly_departed:
            if veh_id in vehicle_data and vehicle_data[veh_id]['actual_depart'] is None:
                vehicle_data[veh_id]['actual_depart'] = current_time
                departed_vehicles.add(veh_id)
        
        # Track arrived vehicles
        newly_arrived = traci.simulation.getArrivedIDList()
        for veh_id in newly_arrived:
            if veh_id in vehicle_data and vehicle_data[veh_id]['actual_arrival'] is None:
                vehicle_data[veh_id]['actual_arrival'] = current_time
                arrived_vehicles.add(veh_id)
        
        # Collect detailed data for each vehicle at this timestep
        for veh_id in vehicle_ids:
            if veh_id not in vehicle_data:
                continue
            
            try:
                # Get vehicle's current edge
                edge_id = traci.vehicle.getRoadID(veh_id)
                
                # Skip internal edges (junctions)
                if edge_id.startswith(':'):
                    continue
                
                # Get waiting time
                waiting_time = traci.vehicle.getWaitingTime(veh_id)
                vehicle_data[veh_id]['wait_time'] = max(vehicle_data[veh_id]['wait_time'], waiting_time)
                
                # Extract from/to nodes from edge ID (e.g., "E1_3" -> from=1, to=3)
                if edge_id.startswith('E'):
                    parts = edge_id[1:].split('_')
                    if len(parts) >= 2:
                        path_from = int(parts[0])
                        path_to = int(parts[1])
                        
                        # Calculate block (stopped vehicles or congestion indicator)
                        # Here we use waiting time as a proxy for blocking
                        block = int(waiting_time)
                        
                        # Record detailed data
                        detailed_records.append({
                            'time': current_time,
                            'player_id': vehicle_data[veh_id]['player_id'],
                            'path_from': path_from,
                            'path_to': path_to,
                            'block': block
                        })
            
            except traci.exceptions.TraCIException:
                # Vehicle might have just left
                pass
        
        step += 1
        if step % 100 == 0:
            print(f".", end='', flush=True)
    
    print(f"\n✓ Simulation completed!")
    print(f"  Total steps: {step}")
    print(f"  Departed vehicles: {len(departed_vehicles)}")
    print(f"  Arrived vehicles: {len(arrived_vehicles)}")
    
    # Close TraCI
    traci.close()
    
    # Generate result_summary.csv
    summary_data = []
    for veh_id, data in vehicle_data.items():
        if data['actual_arrival'] is not None:  # Only include vehicles that completed their trip
            summary_data.append({
                'player_id': data['player_id'],
                'arrived_time': data['destination'],  # Original destination node
                'departed_time': data['departure_time'],  # Original departure time
                'wait_time': int(data['wait_time'])
            })
    
    # Sort by player_id
    summary_data.sort(key=lambda x: x['player_id'])
    
    # Save result_summary.csv
    summary_df = pd.DataFrame(summary_data)
    summary_output = '/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result_sumo/result_summary.csv'
    os.makedirs('result', exist_ok=True)
    summary_df.to_csv(summary_output, index=False)
    print(f"\n✓ Saved {len(summary_data)} vehicle summaries to {summary_output}")
    
    # Generate result_detailed.csv
    detailed_df = pd.DataFrame(detailed_records)
    detailed_df.sort_values(['time', 'player_id'], inplace=True)
    
    detailed_output = '/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result_sumo/result_detailed.csv'
    detailed_df.to_csv(detailed_output, index=False)
    print(f"✓ Saved {len(detailed_records)} detailed records to {detailed_output}")
    
    # Display summary statistics
    print(f"\nSummary Statistics:")
    print(f"  Total vehicles: {len(summary_data)}")
    print(f"  Average wait time: {summary_df['wait_time'].mean():.2f}s")
    print(f"  Max wait time: {summary_df['wait_time'].max()}s")
    print(f"  Detailed records: {len(detailed_records)}")
    
except traci.exceptions.FatalTraCIError as e:
    print(f"\n✗ Fatal TraCI error: {e}")
    print("Check that:")
    print("  1. simulation.sumocfg exists")
    print("  2. Network and route files are valid")
    
except Exception as e:
    print(f"\n✗ Unexpected error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    if traci.isLoaded():
        traci.close()
        print("TraCI connection closed")

Loaded 2000 trips from demand
Starting SUMO simulation...
 Retrying in 1 seconds


Fontconfig warning: "/usr/share/fontconfig/conf.avail/05-reset-dirs-sample.conf", line 6: unknown element "reset-dirs"


Simulation running..................................
✓ Simulation completed!
  Total steps: 3130
  Departed vehicles: 2000
  Arrived vehicles: 2000

✓ Saved 2000 vehicle summaries to /mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result_sumo/result_summary.csv
✓ Saved 1240445 detailed records to /mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result_sumo/result_detailed.csv

Summary Statistics:
  Total vehicles: 2000
  Average wait time: 19.52s
  Max wait time: 103s
  Detailed records: 1240445


In [6]:
data = pd.read_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result_sumo/result_summary.csv")
data_detailed = pd.read_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result_sumo/result_detailed.csv")
data_detailed

,time,player_id,path_from,path_to,block
0,2,1,2,6,0
1,2,2,20,18,0
2,3,1,2,6,0
3,3,2,20,18,0
4,3,3,16,17,0
...,...,...,...,...,...
1240440,3125,1809,6,2,0
1240441,3126,1809,6,2,0
1240442,3127,1809,6,2,0
1240443,3128,1809,6,2,0


In [10]:
# Run comprehensive network analysis
# This will analyze all lanes, generate plots, and save results

analysis = Analysis(data=data, detailed_data=data_detailed)

# Specify output directory (relative to notebook location)
output_directory = "/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/analysis_output_sumo"

# Run full analysis
results = analysis.run_full_network_analysis(output_dir=output_directory)

# Access the results
print("\n📊 Summary of Results:")
print(f"Lane Metrics DataFrame shape: {results['lane_metrics'].shape}")
print(f"Network Stats Keys: {list(results['network_stats'].keys())}")
print(f"Output saved to: {results['output_dir']}")

📁 Output directory: /mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/analysis_output_sumo/analysis_20260104_001403
🔍 Found 76 lanes with vehicle data


Processing lanes:   0%|          | 0/76 [00:00<?, ?it/s]

/tmp/ipykernel_14367/876854491.py:154: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  total_delay = np.trapz(queue_length, time_range)
/tmp/ipykernel_14367/876854491.py:368: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1, 0].boxplot(box_data, labels=['Wait Time', 'Travel Time'], patch_artist=True)
/tmp/ipykernel_14367/876854491.py:581: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans Mono.
  plt.savefig(os.path.join(output_dir, "comprehensive_dashboard.png"), dpi=200, bbox_inches='tight')
/tmp/ipykernel_14367/876854491.py:581: UserWarning: Glyph 128663 (\N{AUTOMOBILE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "comprehensive_dashboard.png"), dpi=200, bbox_inches='tight')


📄 Saved CSV files to /mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/analysis_output_sumo/analysis_20260104_001403/summary

TRAFFIC SIMULATION ANALYSIS REPORT
Generated: 2026-01-04 00:15:06

NETWORK-WIDE STATISTICS
----------------------------------------
Total Trips Completed: 2,000
Simulation Duration: 1797 seconds
Throughput: 4007 vehicles/hour

WAIT TIME STATISTICS
----------------------------------------
Average Wait Time: 19.52 seconds
Median Wait Time: 19.00 seconds
Std Dev Wait Time: 17.79 seconds
Min Wait Time: 0.00 seconds
Max Wait Time: 103.00 seconds
25th Percentile: 0.00 seconds
75th Percentile: 31.00 seconds
95th Percentile: 56.00 seconds

TRAVEL TIME STATISTICS
----------------------------------------
Average Travel Time: 898.43 seconds
Median Travel Time: 901.00 seconds
Std Dev Travel Time: 512.57 seconds
Min Travel Time: -20.00 seconds
Max Travel Time: 1796.00 seconds
25th Percentile: 465.75 seconds
75th Percentile: 1341.25 seconds
95th Percentil